# Ілюстрація `declarations.py`
## Мета
Цей ноутбук створений для того, аби проілюструвати, як працює скрипт.

Розглянуті нижче ф-ції покликані автоматизувати збір інформації про майнові декларації

In [1]:
# завантаження бібліотек
import os
from time import sleep
from typing import Any, List, Iterable

import requests
import pandas as pd
from pandas.io.json import json_normalize

# завантаження функцій
from declarations import (
    save, step, create_dataframe, 
    reshape_dataframe, rm_datedups
)

---

In [2]:
# отримання даних щодо судді з реєстру declarations.com.ua 
person = "Лисенко Вадим Олександрович"
url = f"http://declarations.com.ua/search?q={person}+AND+суддя&deepsearch=on&format=opendata"
r = requests.get(url, sleep(0.5)).json()

In [3]:
path = r["results"]["object_list"]
table = create_dataframe(path) # без інфи по статках, 13 колонок
table.shape

(20, 13)

In [4]:
table

,created_date,declaration_year,document_type,first_name,id,is_corrected,last_name,office,patronymic,position,source,url,link
0,2018-04-10T00:00:00,2018,Форма змін,Вадим,nacp_1392653e-e056-4b64-92ff-e46144c0ea8d,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_1...,https://public-api.nazk.gov.ua/v1/declaration/...
1,2018-06-20T00:00:00,2018,Форма змін,Вадим,nacp_b1d0fe4f-cf92-41ae-a33b-5ff810684daa,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_b...,https://public-api.nazk.gov.ua/v1/declaration/...
2,2018-05-18T00:00:00,2018,Форма змін,Вадим,nacp_a8b5faf0-d60a-43ab-b42b-c28470030467,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_a...,https://public-api.nazk.gov.ua/v1/declaration/...
3,2017-11-29T00:00:00,2017,Форма змін,Вадим,nacp_4fe2f818-d2b6-4181-a03d-126c2ffc263e,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_4...,https://public-api.nazk.gov.ua/v1/declaration/...
4,2018-05-29T00:00:00,2018,Форма змін,Вадим,nacp_3db209d0-5e0e-4bdb-888c-528e5d1703e8,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_3...,https://public-api.nazk.gov.ua/v1/declaration/...
5,2019-05-31T00:00:00,2019,Форма змін,Вадим,nacp_7eb9897e-41e0-476e-8742-fc051add1183,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_7...,https://public-api.nazk.gov.ua/v1/declaration/...
6,2019-09-21T00:00:00,2019,Форма змін,Вадим,nacp_3c8b7440-48cf-4909-9dbc-20b19ac4820c,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_3...,https://public-api.nazk.gov.ua/v1/declaration/...
7,2018-04-10T00:00:00,2018,Форма змін,Вадим,nacp_7e13437e-32da-49d3-99ff-de0787600a5a,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_7...,https://public-api.nazk.gov.ua/v1/declaration/...
8,2019-08-01T00:00:00,2019,Форма змін,Вадим,nacp_0f090ca5-185e-4a14-8db7-5635fba5025d,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_0...,https://public-api.nazk.gov.ua/v1/declaration/...
9,2018-05-25T00:00:00,2018,Форма змін,Вадим,nacp_d1ab2654-1ffc-4073-a40f-8470fe813071,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_d...,https://public-api.nazk.gov.ua/v1/declaration/...


In [5]:
table = create_dataframe(r["results"]["object_list"], detailed=True) # з інфою по статках
table.shape

(20, 16)

In [6]:
table

,created_date,declaration_year,document_type,first_name,id,is_corrected,last_name,office,patronymic,position,source,url,link,gift_num,property_num,vehicle_num
0,2018-04-10T00:00:00,2018,Форма змін,Вадим,nacp_1392653e-e056-4b64-92ff-e46144c0ea8d,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_1...,https://public-api.nazk.gov.ua/v1/declaration/...,0,0,0
1,2018-06-20T00:00:00,2018,Форма змін,Вадим,nacp_b1d0fe4f-cf92-41ae-a33b-5ff810684daa,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_b...,https://public-api.nazk.gov.ua/v1/declaration/...,0,1,0
2,2018-05-18T00:00:00,2018,Форма змін,Вадим,nacp_a8b5faf0-d60a-43ab-b42b-c28470030467,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_a...,https://public-api.nazk.gov.ua/v1/declaration/...,0,0,0
3,2017-11-29T00:00:00,2017,Форма змін,Вадим,nacp_4fe2f818-d2b6-4181-a03d-126c2ffc263e,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_4...,https://public-api.nazk.gov.ua/v1/declaration/...,0,0,0
4,2018-05-29T00:00:00,2018,Форма змін,Вадим,nacp_3db209d0-5e0e-4bdb-888c-528e5d1703e8,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_3...,https://public-api.nazk.gov.ua/v1/declaration/...,0,0,0
5,2019-05-31T00:00:00,2019,Форма змін,Вадим,nacp_7eb9897e-41e0-476e-8742-fc051add1183,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_7...,https://public-api.nazk.gov.ua/v1/declaration/...,0,0,0
6,2019-09-21T00:00:00,2019,Форма змін,Вадим,nacp_3c8b7440-48cf-4909-9dbc-20b19ac4820c,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_3...,https://public-api.nazk.gov.ua/v1/declaration/...,0,0,0
7,2018-04-10T00:00:00,2018,Форма змін,Вадим,nacp_7e13437e-32da-49d3-99ff-de0787600a5a,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_7...,https://public-api.nazk.gov.ua/v1/declaration/...,0,0,0
8,2019-08-01T00:00:00,2019,Форма змін,Вадим,nacp_0f090ca5-185e-4a14-8db7-5635fba5025d,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_0...,https://public-api.nazk.gov.ua/v1/declaration/...,0,0,0
9,2018-05-25T00:00:00,2018,Форма змін,Вадим,nacp_d1ab2654-1ffc-4073-a40f-8470fe813071,False,Лисенко,Амур-Нижньодніпровський районний суд міста Дні...,Олександрович,суддя,NACP,https://declarations.com.ua/declaration/nacp_d...,https://public-api.nazk.gov.ua/v1/declaration/...,0,1,0


In [7]:
reshaped_df = reshape_dataframe(table, person)
reshaped_df.shape

(10, 10)

In [8]:
reshaped_df

,created_date,declaration_year,document_type,office,source,link,gift_num,property_num,vehicle_num,name
10,2019-09-18T00:00:00,2018,Кандидата на посаду,Дніпровський апеляційний суд,NACP,https://public-api.nazk.gov.ua/v1/declaration/...,0,5,1,Лисенко Вадим Олександрович
11,2017-03-16T00:00:00,2016,Щорічна,Амур-Нижньодніпровський районний суд міста Дні...,NACP,https://public-api.nazk.gov.ua/v1/declaration/...,0,2,2,Лисенко Вадим Олександрович
12,2016-10-28T00:00:00,2015,Щорічна,Амур-Нижньодніпровський районний суд міста Дні...,NACP,https://public-api.nazk.gov.ua/v1/declaration/...,0,1,2,Лисенко Вадим Олександрович
13,2016-10-25T00:00:00,2015,Щорічна,Амур-Нижньодніпровський районний суд міста Дні...,NACP,https://public-api.nazk.gov.ua/v1/declaration/...,0,1,2,Лисенко Вадим Олександрович
14,2017-03-16T00:00:00,2016,Щорічна,Амур-Нижньодніпровський районний суд міста Дні...,NACP,https://public-api.nazk.gov.ua/v1/declaration/...,0,1,2,Лисенко Вадим Олександрович
15,2018-02-26T00:00:00,2017,Щорічна,Амур-Нижньодніпровський районний суд міста Дні...,NACP,https://public-api.nazk.gov.ua/v1/declaration/...,0,6,3,Лисенко Вадим Олександрович
16,2019-03-17T00:00:00,2018,Щорічна,Амур-Нижньодніпровський районний суд міста Дні...,NACP,https://public-api.nazk.gov.ua/v1/declaration/...,0,5,1,Лисенко Вадим Олександрович
17,2019-03-12T00:00:00,2018,Щорічна,Амур-Нижньодніпровський районний суд міста Дні...,NACP,https://public-api.nazk.gov.ua/v1/declaration/...,0,5,1,Лисенко Вадим Олександрович
18,None,2014,Щорічна,Загальний місцевий суд,VULYK,http://static.declarations.com.ua/declarations...,0,0,0,Лисенко Вадим Олександрович
19,None,2015,None,Загальний місцевий суд,VULYK,http://static.declarations.com.ua/declarations...,0,0,0,Лисенко Вадим Олександрович


In [9]:
frame = rm_datedups(reshaped_df)
frame.shape

(5, 10)

In [10]:
frame

,created_date,declaration_year,document_type,office,source,link,gift_num,property_num,vehicle_num,name
18,NaT,2014,Щорічна,Загальний місцевий суд,VULYK,http://static.declarations.com.ua/declarations...,0,0,0,Лисенко Вадим Олександрович
12,2016-10-28,2015,Щорічна,Амур-Нижньодніпровський районний суд міста Дні...,NACP,https://public.nazk.gov.ua/declaration/4267d4b...,0,1,2,Лисенко Вадим Олександрович
14,2017-03-16,2016,Щорічна,Амур-Нижньодніпровський районний суд міста Дні...,NACP,https://public.nazk.gov.ua/declaration/8fde0ea...,0,1,2,Лисенко Вадим Олександрович
15,2018-02-26,2017,Щорічна,Амур-Нижньодніпровський районний суд міста Дні...,NACP,https://public.nazk.gov.ua/declaration/de89f56...,0,6,3,Лисенко Вадим Олександрович
10,2019-09-18,2018,Кандидата на посаду,Дніпровський апеляційний суд,NACP,https://public.nazk.gov.ua/declaration/e5cfcbc...,0,5,1,Лисенко Вадим Олександрович


In [12]:
def pseudo_save(df):
    """
    Ілюструє, як виглядає результат перед збереженням
    """
    
    df["paste"] = df["declaration_year"].astype(str).str.cat(df["link"], sep="| ")
    df = df[["paste", "gift_num", "property_num", "vehicle_num"]]
    df.columns = ["Скопіювати", "Подарунок", "Нерухоме", "Рухоме"]
    return df

In [13]:
pseudo_save(frame)

,Скопіювати,Подарунок,Нерухоме,Рухоме
18,2014| http://static.declarations.com.ua/declar...,0,0,0
12,2015| https://public.nazk.gov.ua/declaration/4...,0,1,2
14,2016| https://public.nazk.gov.ua/declaration/8...,0,1,2
15,2017| https://public.nazk.gov.ua/declaration/d...,0,6,3
10,2018| https://public.nazk.gov.ua/declaration/e...,0,5,1


---

чим є `r`:

In [12]:
print(type(r))
print(r.keys())
print(r["results"].keys()) # ключі всередині ключа results

<class 'dict'>
dict_keys(['query', 'meta', 'deepsearch', 'results'])
dict_keys(['paginator', 'number', 'object_list'])


In [14]:
# одна декларація у машиночитаному форматі
r["results"]["object_list"][0]

{'guid': 'nacp_1392653e-e056-4b64-92ff-e46144c0ea8d',
 'infocard': {'first_name': 'Вадим',
  'patronymic': 'Олександрович',
  'last_name': 'Лисенко',
  'office': 'Амур-Нижньодніпровський районний суд міста Дніпропетровська',
  'position': 'суддя',
  'source': 'NACP',
  'id': 'nacp_1392653e-e056-4b64-92ff-e46144c0ea8d',
  'url': 'https://declarations.com.ua/declaration/nacp_1392653e-e056-4b64-92ff-e46144c0ea8d',
  'document_type': 'Форма змін',
  'is_corrected': False,
  'created_date': '2018-04-10T00:00:00',
  'declaration_year': 2018},
 'raw_source': {'url': 'https://public-api.nazk.gov.ua/v1/declaration/1392653e-e056-4b64-92ff-e46144c0ea8d'},
 'unified_source': {'step_0': {'changesYear': '2018'},
  'step_1': {'firstname': 'Вадим',
   'lastname': 'Лисенко',
   'middlename': 'Олександрович',
   'postCategory': '',
   'postCategory_extendedstatus': '1',
   'postType': '',
   'postType_extendedstatus': '1',
   'workPlace': 'Амур-Нижньодніпровський районний суд міста Дніпропетровська',
  

Кожна декларація має поля `infocard` `raw_source` та `unified_source`

In [15]:
r["results"]["object_list"][0]["infocard"]

{'first_name': 'Вадим',
 'patronymic': 'Олександрович',
 'last_name': 'Лисенко',
 'office': 'Амур-Нижньодніпровський районний суд міста Дніпропетровська',
 'position': 'суддя',
 'source': 'NACP',
 'id': 'nacp_1392653e-e056-4b64-92ff-e46144c0ea8d',
 'url': 'https://declarations.com.ua/declaration/nacp_1392653e-e056-4b64-92ff-e46144c0ea8d',
 'document_type': 'Форма змін',
 'is_corrected': False,
 'created_date': '2018-04-10T00:00:00',
 'declaration_year': 2018}

In [16]:
# декларації >= 2015; цікавить `url`
r["results"]["object_list"][0]["raw_source"]

{'url': 'https://public-api.nazk.gov.ua/v1/declaration/1392653e-e056-4b64-92ff-e46144c0ea8d'}

In [19]:
# декларації < 2015; цікавить `url`
r["results"]["object_list"][18]["raw_source"]["declaration"]

{'additional_info': False,
 'additional_info_text': '',
 'date': '2015-03-25T00:00:00',
 'notfull': False,
 'notfull_lostpages': '',
 'url': 'http://static.declarations.com.ua/declarations/chosen_ones/mega_batch/lysenko_vadym_oleksandrovych.pdf'}

In [20]:
r["results"]["object_list"][0]["unified_source"]

{'step_0': {'changesYear': '2018'},
 'step_1': {'firstname': 'Вадим',
  'lastname': 'Лисенко',
  'middlename': 'Олександрович',
  'postCategory': '',
  'postCategory_extendedstatus': '1',
  'postType': '',
  'postType_extendedstatus': '1',
  'workPlace': 'Амур-Нижньодніпровський районний суд міста Дніпропетровська',
  'workPost': 'суддя',
  'dnt_organization_group': 'Суд'},
 'step_2': {'1523000017873': {'incomeSource': 'j',
   'iteration': '1523000017873',
   'objectType': 'Спадщина',
   'otherObjectType': '',
   'owningDate': '04.04.2018',
   'person': '1',
   'sizeIncome': '225567',
   'source_citizen': 'Громадянин України',
   'source_eng_company_address': '',
   'source_eng_company_code': '',
   'source_eng_company_name': '',
   'source_eng_firstname': '',
   'source_eng_lastname': '',
   'source_eng_middlename': '',
   'source_eng_sameRegLivingAddress': '',
   'source_ua_company_code': '',
   'source_ua_company_name': '',
   'source_ua_firstname': 'Олександр',
   'source_ua_lastna

In [21]:
# у формі змін
r["results"]["object_list"][0]["unified_source"].keys() 

dict_keys(['step_0', 'step_1', 'step_2', 'step_4'])

In [23]:
# у звичайних
r["results"]["object_list"][10]["unified_source"].keys() 

dict_keys(['step_0', 'step_1', 'step_11', 'step_12', 'step_16', 'step_2', 'step_3', 'step_6'])

In [24]:
r["results"]["object_list"][10]["unified_source"]["step_11"]

{'1475753774688': {'incomeSource': 'j',
  'iteration': '1475753774688',
  'objectType': 'Заробітна плата отримана за основним місцем роботи',
  'otherObjectType': '',
  'person': '1',
  'rights': {'1': {'citizen': '',
    'eng_company_address': '',
    'eng_company_code': '',
    'eng_company_name': '',
    'eng_firstname': '',
    'eng_fullname': '',
    'eng_lastname': '',
    'eng_middlename': '',
    'eng_middlename_extendedstatus': '',
    'eng_postCode': '',
    'eng_postCode_extendedstatus': '',
    'otherOwnership': '',
    'ownershipType': 'Власність',
    'percent-ownership': '',
    'postCode': '[Конфіденційна інформація]',
    'rightBelongs': '1',
    'rights_cityPath': '',
    'rights_id': 1,
    'ua_apartmentsNum_extendedstatus': '',
    'ua_city': '',
    'ua_company_code': '',
    'ua_company_name': '',
    'ua_firstname': '',
    'ua_houseNum_extendedstatus': '',
    'ua_housePartNum_extendedstatus': '',
    'ua_lastname': '',
    'ua_middlename': '',
    'ua_middlenam